# Fluorescence lifetime analysis
This notebook will help guide you through the analysis for the experiment "Laser Induced Fluorescence of Quinine Sulfate and Kinetics of Cl$^-$ Induced Fluorescence Quenching" based on the experiment by Gutow (*J. Chem. Ed.*, **2005**). You will use the curve fitting and plotting skills that we learned in the first week of lab. 

We start by importing the libraries we need, then uploading the data files (if using Google Colab) and extracting the data into a Numpy array.

In the first section, you will define a simple exponential function to fit to your data. Try this for a few of your experiments and see how well this fit works.

In the second section, you will use a function `full_fit`, which allows us to account for the instrument response function in our data. Does this improve the quality of our fits? Which experimental conditions require this more sophisticated fit?

In the last section, you will programatically fit all of your data and extract the fluorescence lifetimes. You will then fit the fluorescence decay rates vs [Cl$^-$] to determine the mechanism for fluorescence quenching in this system.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.signal import convolve 
from scipy.fftpack import next_fast_len 
import scipy.optimize

In [ ]:
### Import the data files to analyze
### Skip this if you are using a local installation of Jupyter
from google.colab import files
uploaded=files.upload()

Saving IRF1.txt to IRF1.txt


In [ ]:
### Use the np.genfromtext() function to extract data from files and place in a Numpy array, 
### use the argument "unpack=True" to transpose the data for easier access 

### Simple fit to the data
You will define a basic exponential decay function here and attempt to fit some of your lifetime data. The `idx` variable defined below may be helpful if you don't want to fit your data starting from the very first time point collected.

In [ ]:
### Define a fit function for your data 

In [ ]:
### You may want to fit only a portion of your data, defining a start and end
### point here gives you an "idx" so you can use data[idx] when calling curve_fit
### this assumes your time variable is called "t"
start=160
end=230
idx = [i for i,v in enumerate(t) if v>start and v < end]

In [ ]:
### Call scipy.optimize.curve_fit() to fit your data, you may want to give the
### curve_fit some initial guesses by passing the argument p0=[] which contains
### intial guess values for your parameters.

In [ ]:
### Calculate the standard deviation of your fit parameters from the covariance matrix
stdev = np.sqrt(np.diag(pcov))

In [ ]:
### Plot the data and your fit to see if it makes sense!


### You may want to write out the lifetime directly on your plot. Use the function 
### plt.text(xcoord, ycoord, f"{popt[i]:.1f} ns") to programatically plot the fit
### value. You need to provide the xcoord and ycoord location for the text and 
## popt[i] is the fitted parameter corresponding to the lifetime

### Sophisticated fit to the data, which accounts for the instrument response function
Add a call to the function that you defined above into the definition for `full_fit` where you see WORK HERE. Be sure that the arguments for your function agree with the arguments `tau`, `a`, and `offset` for `full_fit`.

In [ ]:
### Use the np.genfromtext() function to extract the instrument response function
### from your file and place in a Numpy array called "irf"
### use the argument "unpack=True" to transpose the data for easier access 

In [ ]:
### function to fit data with IRF convolution
def full_fit(t,tau,a,b,offset):

    ''' tau : lifetime
        a : amplitude of exponential decay
        b : amplitude of IRF
        offset : vertical offset in case signal does not go all the way to zero
    '''
       
    ### WORK HERE ###
    y = 'name of your function with correct arguments'

    N=10000 # padding zeros
    y_pad = np.concatenate((y, np.zeros(N))) # zero pad exponential decay
    irf_pad = np.concatenate((irf, np.zeros(N))) # zero pad IRF

    f_y = np.fft.fft(y_pad) # fft padded exponential decay
    f_irf = np.fft.fft(b * irf_pad) # fft padded IRF times weighting factor

    C = f_y * f_irf # convolution in time domain is multiplication in frequency domain

    y_full = np.fft.ifft(C) # inverse fft
    y_fit = y_full[0:len(t)] # trim zero pad
    y_fit = np.real(y_fit) # real part only

    return y_fit

In [ ]:
### Call scipy.optimize.curve_fit() to fit your data, you may want to give the
### curve_fit some initial guesses by passing the argument p0=[] which contains
### intial guess values for your parameters.

In [ ]:
### Calculate the standard deviation of your fit parameters from the covariance matrix
stdev = np.sqrt(np.diag(pcov))

In [ ]:
### Plot the data and your fit to see if it makes sense! Include the fit from your 
### simple function above to see the improvement.


### You may want to write out the lifetime directly on your plot. Use the function 
### `plt.text(xcoord, ycoord, f"{popt[i]:.1f} ns")` to programatically plot the fit
### value. You need to provide the xcoord and ycoord location for the text and 
## popt[i] is the fitted parameter corresponding to the lifetime

### Fit all data and then plot fluorescence decay rates as a function of chloride concentration

In the cell immediately below is an example of how we might use a **for** loop to sequentially load each data file and fit everything in just a few lines of code. Modify this code to work with your data!

In the last cell, you will plot your data as decay rate vs [Cl$^-$], along with the results of a linear fit. We would like to show error bars on the data points for decay rate using the function `plt.errorbar(xdata, ydata, yerr)`. We have to be careful here! The errors that we obtain from our fit to the raw data are the errors in the fitted lifetime ($\tau$), NOT the errors in the fitted rate ($k=1/\tau$). We should use the rules for error propagation to obtain the correct error. Recall for division/multiplication:
\begin{equation}
\frac{\delta z}{z}=\frac{\delta x}{x} + \frac{\delta y}{y}
\end{equation}
In our situation, $k=1/\tau$, so to calculate the error for $k$:
\begin{equation}
\frac{\delta k}{k} = \frac{\delta \tau}{\tau} \\
\delta k = k \times \frac{\delta \tau}{\tau} \\
\delta k = \frac{\delta \tau}{\tau^2}
\end{equation}

In [ ]:
### this is just an example of how you might loop over a series of data files, 
### fit each one, and save the results in an array. change this to work for the
### names of your data files and fit function

tau = np.array([])
err = np.array([])

for i in range(1,5): # range(1,5) means [1, 2, 3, 4] since python index is off by 1
  filename = "quinine_sample"+str(i)+"_lifetime.txt"
  t, data = np.genfromtxt(filename,skip_header=4,skip_footer=1,unpack=True)
  popt, pcov = scipy.optimize.curve_fit(full_fit, t, data, p0=[700, 20, 1, 20])
  stdev = np.sqrt(np.diag(pcov))
  tau = np.append(tau,popt[0])
  err = np.append(err,stdev[0])

In [ ]:
### define an array with your chloride ion concentrations

In [ ]:
### perform a linear fit of the rates (1/lifetime) vs chloride concentration.
### you can either define a function and use `curve_fit` like above or use the 
### `np.polyfit` function. To get the covariance matrix out of `np.polyfit`, you
### must add the arguments 'full=False, cov=True' to the function call

In [ ]:
### Calculate the standard deviation of your fit parameters from the covariance matrix
stdev = np.sqrt(np.diag(pcov))

In [ ]:
### Plot the data and your fit to see if it makes sense! On this plot, we would
### like to include error bars on each data point, which come from the 'stdev' of
### your fit calculated in the cell above. Use the function `plt.errorbar(xdata, ydata, yerr)`
### where ydata is the rate and yerr is the error of the rate (NOT the error of the lifetime)


### You may want to write out the lifetime directly on your plot. Use the function 
### plt.text(xcoord, ycoord, f"{popt[i]:.1f} ns") to programatically plot the fit
### value. You need to provide the xcoord and ycoord location for the text and 
## popt[i] is the fitted parameter corresponding to the lifetime